In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

!pip install transformers
import transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.1 MB/s eta 0:00:00


In [ ]:
!unzip /content/ner_dataset.csv.zip

Archive:  /content/ner_dataset.csv.zip
  inflating: ner_dataset.csv         


In [ ]:
df = pd.read_csv("/content/ner_dataset.csv",encoding = 'ISO-8859-1')
df = df.dropna()
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
24,Sentence: 2,Families,NNS,O
54,Sentence: 3,They,PRP,O
68,Sentence: 4,Police,NNS,O
83,Sentence: 5,The,DT,O


In [ ]:
df.shape

(47959, 4)

In [ ]:
df.Tag.unique()


array(['O', 'B-gpe', 'B-geo', 'B-per', 'B-org', 'B-tim', 'B-nat', 'B-eve',
       'B-art'], dtype=object)

In [ ]:
len(df.Tag.unique())

9

In [ ]:
pie = df['Tag'].value_counts()
pie

O        34310
B-per     4019
B-geo     3335
B-gpe     2989
B-org     2752
B-tim      515
B-art       18
B-nat       11
B-eve       10
Name: Tag, dtype: int64

In [ ]:
pie = df['Tag'].value_counts()
px.pie(names = pie.index,values= pie.values,hole = 0.5,title ='Total Count of Tags')

In [ ]:
len(df['POS'].value_counts())

30

In [ ]:
bar = df['POS'].value_counts()
px.bar(x = bar.index,y = bar.values,title = "Total Count of Parts of Speech",color = bar.index)

In [ ]:
df.isna().sum()

Sentence #    0
Word          0
POS           0
Tag           0
dtype: int64

In [ ]:
sentence = df.groupby("Sentence #")["Word"].apply(list).values

In [ ]:
sentence

array([list(['Thousands']), list(['Iranian']), list(['Helicopter']), ...,
       list(['Following']), list(['Since']), list(['The'])], dtype=object)

In [ ]:
len(sentence)

47959

In [ ]:
pos = df.groupby(by = 'Sentence #')['POS'].apply(list).values
pos


array([list(['NNS']), list(['JJ']), list(['NN']), ..., list(['VBG']),
       list(['IN']), list(['DT'])], dtype=object)

In [ ]:
len(pos)

47959

In [ ]:
tag = df.groupby(by = 'Sentence #')['Tag'].apply(list).values
tag

array([list(['O']), list(['B-gpe']), list(['O']), ..., list(['O']),
       list(['O']), list(['O'])], dtype=object)

In [ ]:
len(tag)

47959

In [ ]:
def process_data(df):

    enc_pos = preprocessing.LabelEncoder()
    enc_tag = preprocessing.LabelEncoder()

    df.loc[:, "POS"] = enc_pos.fit_transform(df["POS"])
    df.loc[:, "Tag"] = enc_tag.fit_transform(df["Tag"])

    sentences = df.groupby("Sentence #")["Word"].apply(list).values
    pos = df.groupby("Sentence #")["POS"].apply(list).values
    tag = df.groupby("Sentence #")["Tag"].apply(list).values
    return sentences, pos, tag, enc_pos, enc_tag

sentence,pos,tag,enc_pos,enc_tag = process_data(df)

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
MAX_LEN = 128
def tokenize(data,max_len = MAX_LEN):
    input_ids = list()
    attention_mask = list()
    for i in tqdm(range(len(data))):
        print(data[i])
        #encode_plus returns a dictionary containing the encoded sequence or sequence pair and additional information: the mask for sequence classification and the overflowing elements if a max_length is specified.
        #https://huggingface.co/transformers/v2.11.0/main_classes/tokenizer.html
        encoded = tokenizer.encode_plus(data[i],
                                        add_special_tokens = True,
                                        max_length = MAX_LEN,
                                        is_split_into_words=True,
                                        return_attention_mask=True,
                                        padding = 'max_length',
                                        truncation=True,return_tensors = 'np')

        print(input_ids)
        print(attention_mask)
        input_ids.append(encoded['input_ids'])
        attention_mask.append(encoded['attention_mask'])
    return np.vstack(input_ids),np.vstack(attention_mask)


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(sentence,pos,random_state=42,test_size=0.1)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((43163,), (4796,), (43163,), (4796,))

In [ ]:
input_ids,attention_mask = tokenize(X_train,max_len = MAX_LEN)

100%|██████████| 43163/43163 [00:06<00:00, 7119.91it/s]


In [ ]:
val_input_ids,val_attention_mask = tokenize(X_test,max_len = MAX_LEN)


100%|██████████| 4796/4796 [00:00<00:00, 10152.65it/s]


In [ ]:
attention_mask[0]

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# TEST: Checking Padding and Truncation length's
was = list()
for i in range(len(input_ids)):
    was.append(len(input_ids[i]))
set(was)

{128}

In [ ]:
# Train Padding
test_tag = list()
for i in range(len(y_test)):
    test_tag.append(np.array(y_test[i] + [0] * (128-len(y_test[i]))))

# TEST:  Checking Padding Length
was = list()
for i in range(len(test_tag)):
    was.append(len(test_tag[i]))
set(was)

{128}

In [ ]:
# Train Padding
train_tag = list()
for i in range(len(y_train)):
    train_tag.append(np.array(y_train[i] + [0] * (128-len(y_train[i]))))

# TEST:  Checking Padding Length
was = list()
for i in range(len(train_tag)):
    was.append(len(train_tag[i]))
set(was)

{128}

In [ ]:
def create_model(bert_model,max_len = MAX_LEN):
    input_ids = tf.keras.Input(shape = (max_len,),dtype = 'int32')
    attention_masks = tf.keras.Input(shape = (max_len,),dtype = 'int32')
    #https://huggingface.co/docs/transformers/model_doc/bert#transformers.TFBertModel
    bert_output = bert_model(input_ids,attention_mask = attention_masks,return_dict =True)
    embedding = tf.keras.layers.Dropout(0.3)(bert_output["last_hidden_state"])
    output = tf.keras.layers.Dense(30,activation = 'softmax')(embedding)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = [output])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased') #https://github.com/huggingface/transformers/blob/main/src/transformers/models/bert/modeling_tf_bert.py
model = create_model(bert_model,MAX_LEN)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
early_stopping = EarlyStopping(mode='min',patience=5)
history_bert = model.fit([input_ids,attention_mask],np.array(train_tag),validation_data = ([val_input_ids,val_attention_mask],np.array(test_tag)),epochs = 25,batch_size = 30*2,callbacks = early_stopping,verbose = True)